In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install sparkmagic
!pip install pyspark

In [ ]:
spark

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler, OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import countDistinct

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
df = spark.read.csv('../input/cardiovascular-study-dataset-predict-heart-disea/train.csv', inferSchema=True, header=True)
df.show(10)

In [ ]:
df.printSchema()

In [ ]:
columns_to_drop = ['id', 'education']
df = df.drop(*columns_to_drop)

In [ ]:
train, valid = df.randomSplit([0.8, 0.2], seed = 7) 
train_rows = train.count()
test_rows = valid.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

In [ ]:
train.dtypes

In [ ]:
catCols = [x for [x, dataType] in train.dtypes if dataType == 'string']
numCols = [x for [x, dataType] in train.dtypes if (((dataType == 'double') | (dataType == 'int')) & (x != 'TenYearCHD'))]

In [ ]:
print(catCols)
print(numCols)

In [ ]:
train.agg(countDistinct('sex')).show()

In [ ]:
train.agg(countDistinct('is_smoking')).show()

In [ ]:
train.groupBy('sex').count().show()

In [ ]:
train.groupBy('is_smoking').count().show()

In [ ]:
string_indexer = [StringIndexer(inputCol = x, outputCol = x+"_stringIdexer", handleInvalid = 'skip') for x in catCols]

In [ ]:
string_indexer

In [ ]:
one_hot_encoder = [OneHotEncoder(inputCols=[f"{x}_stringIndexer" for x in catCols], outputCols=[f"{x}_OneHotEncoder" for x in catCols])]

In [ ]:
one_hot_encoder

In [ ]:
assemblerInput = [x for x in numCols]
assemblerInput += [f"{x}_OneHotEncoder" for x in catCols]

In [ ]:
assemblerInput

In [ ]:
vector_assembler = VectorAssembler(inputCols=assemblerInput ,outputCol="VectorAssembler_features")

In [ ]:
stages = []
stages += string_indexer
stages += one_hot_encoder
stages += [vector_assembler]

In [ ]:
stages

In [ ]:
%%time
pipeline = Pipeline().setStages(stages)
model = pipeline.fit(train)

pp_df = model.transform(valid)